Before you turn in this assignment, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then run the test cells for each of the questions you have answered.  Note that a grade of 3 for the related LOs requires all tests in the "Basic Functionality" section to be passed.  The test cells pass if they execute with no errors (i.e. all the assertions are passed).

Make sure you fill in any place that says `YOUR CODE HERE`.  Be sure to remove the `raise NotImplementedError()` statements as you implement your code - these are simply there as a reminder if you forget to add code where it's needed.

---

<h1>CS152 Assignment 2: The DPLL Algorithm</h1>

<h1>Question 1</h1>

Define your <code>Literal</code> class below.  Ensure that you define a <code>name</code> and <code>sign</code> attribute as required in the assignment description.  In addition, include any other attributes and member functions as needed. You will need to overload <code>__neg__()</code> to correctly handle negated literals.  You may need to overload <code>__eq__()</code> and <code>__hash__()</code> also.

**Side notes:** I implemented all the extended functions into the DPLL. The functions work properly, but return a more suitable outputs for my DPLL (e.g. unit clauses and pure symbols heuristics return a sorted list based on the degree heuristic instead of a single symbol). 

I code the extended functions for my DPLL (sort_symbol, deg_heuristic, unit_clause, pure_symbol) and 3 functions with exact same logic but return different type of output to pass the test cases (degree_heuristic, find_unit_clause, find_pure_symbol).

In [0]:
import operator
import copy

In [0]:
# Import any packages you need here
# Also define any variables as needed

# YOUR CODE HERE (OPTIONAL)

class Literal:
  # initialize the Literal with name and sign
  def __init__(self, name, sign = True):
    self.name = name
    self.sign = sign

  # define negative Literal with same name but sign = False
  def __neg__(self):
    return Literal(self.name, not self.sign)

  # hash it so we can put it in dictionary
  def __hash__(self):
    return hash(self.name)

  # equal: same literal name (ignore the sign)
  def __eq__(self, other):
    return self.name == other.name

  # str() and repr in case we want to print them
  def __str__(self):
    # positive sign
    if self.sign:
      return self.name
    # negative sign
    else:
      return "-" + self.name

  def __repr__(self):
    return str(self)

<h1>Question 2</h1>
Implement DPLL by filling in the stubs below.  Note that the various heuristics are not required to be implemented for basic functionality, but a template has been provided for you to do so if you attempt the extension questions

In [0]:
def get_symbol(KB):
  """
  x = get_symbol(KB):
  Take in the KB and return the symbols in the KB

      KB: the input knowledge base
      X: the set of symbols in the KB
  """
  unique_symbol = set()
  # loop through the KB
  for clause in KB:
    for symbol in clause:
      # Append the Symbol and assign them positive sign
      if symbol not in unique_symbol:
        # positive sign
        if symbol.sign:
          unique_symbol.add(symbol)
        # negative sign --> we add -symbol to make it positive
        else:
          unique_symbol.add(-symbol)
  return unique_symbol


def DPLL_Satisfiable(KB):
  global objects
  ''' satisfiable, model = DPLLSatisfiable(KB)
      Takes in a KB and returns whether the KB is satisfiable, and the model that makes it correct
      
      KB: A knowledge base of clauses (CNF) consisting of a list of sets of literals.  A KB might look like
          [{A,B},{-A,C,D}]
      satisfiable: Returns True if the KB is satisfiable, or False otherwise
      Model: A dictionary that assigns a truth value to each literal for the model that satisfies KB.
          For example, a model might look like {A: True, B: False}
  '''
  # YOUR CODE HERE
  # get uniqye symbols
  unique_symbol = get_symbol(KB)
  # put in frozen set so that we can hash them into dictionary
  clauses = [frozenset(clause) for clause in KB]
  model = dict()
  for obj in objects:
    model[obj] = "NA"
  # run function DPLL
  return DPLL(clauses, unique_symbol, model)


def DPLL(clauses, symbols, model):
  global objects, unit_clauses, pure_symbols, sorted_symbols
  ''' satisfiable, model = DPLL(clauses, symbols, model)
      Takes in a KB and returns whether the KB is satisfiable, and the model that makes it correct
      
      clauses: a set of clause from the KB
      symbols: the set of objects
      model: current model 
      satisfiable: Returns True if the KB is satisfiable, or False otherwise
      Model: A dictionary that assigns a truth value to each literal for the model that satisfies KB.
          For example, a model might look like {A: True, B: False}
  '''

  # to kep track of truth_values for all clauses
  truth_values = []
  truth_clauses = {}

  # Looping through the clauses
  for clause in clauses:
    # Use a set to store the truth value of symbols in this clause (for faster look up)
    clause_val = set({})

    """
    "NA stands for free variables or not yet determined"
    Check each symbol:
    - If it's in the model as "NA" then add to the clause_val "NA"
    - If the symbol in the clause is positive, then we add the model[symbol], which 
      is the truth value of that literal in out existing model
      If we find one True value -> that clause will be True -> break
    - If the symbol in the clause is positive, then we add the opposite of model[symbol], 
      which is the opposite truth value of that literal in out existing model
      If we find one True value -> that clause will be True -> break
    - If not in model, add them to model as "NA"
    """
    for symbol in clause:
      if model[symbol] == "NA":
        clause_val.add("NA")
      elif symbol.sign: # if Positive Literal
        clause_val.add(model[symbol])
        if model[symbol] == True:
          break
      else: # if Negative Literal
        if model[symbol] == True: # - True = False
          clause_val.add(False)
        elif model[symbol] == False: # - False = True
          clause_val.add(True) 
          break
    
    # store the truth value of this clause to the main storage
    truth_clauses[clause] = clause_val
    
    # if exist 1 True value -> the whole clause is True
    if True in clause_val:
      truth_values.append(True)
    # If no True value but exist "NA", then it's undetermined 
    elif "NA" in clause_val:
      truth_values.append("NA")
    # this means all symbols are False -> clause is unsatisfiable -> KB Failed
    else: # all False
      # cannot satisfy the model -> not satisfiable
      return False, {}


  # If all the clauses are True, then for the free Literal, we can simply assign them True
  if all(truth_value == True for truth_value in truth_values):
    for obj in objects:
      if model[obj] == 'NA':
        model[obj] = True
    return True, model


  else: # some undetermined clauses
  # Implement all the heuristics

    # Unit Clause Heuristic
    # If unit clause = -1, means that there's a comflict in unit clause: e.g. {C} and {-C}
    # This means KB not satisfiable
    if unit_clauses == -1:
      return False, {}
    # assign all the unit clause to be True
    while unit_clauses:
      obj = unit_clauses.pop()
      model[obj] = obj.sign
    
    # Pure symbol heuristic
    # Make all the pure symbols to be True
    while pure_symbols:
      # pop(-1) because this is based on the ordering of degree heuristic: ascending appearance
      obj = pure_symbols.pop(-1) 
      model[obj] = obj.sign

    # If ran out of symbols: Then KB failed
    if len(symbols) == 0: 
      return False, {}
    
    # Degree Heuristics
    # Pop the most appeared Literal
    pop, remain = deg_heuristic(symbols)

    # Model 1: The most appeared Literal is True
    remain_1 = copy.deepcopy(remain)
    model_1 = copy.deepcopy(model)
    model_1[pop] = True

    # Model 1: The most appeared Literal is False
    remain_2 = copy.deepcopy(remain)
    model_2 = copy.deepcopy(model)
    model_2[pop] = False

    # Run both possible models
    result_1 = DPLL(clauses, remain_1, model_1)
    result_2 = DPLL(clauses, remain_2, model_2)
    
    # whatever model is True: output
    # If no model is True: return False, {}
    if result_1[0]:
      return True, result_1[1]
    if result_2[0]:
      return True, result_2[1]
    return False, {}


def sort_symbol(KB):
  ''' ordered_list = sort_symbol(KB)
      Takes in a KB and returns the ordered_list with symbols order by the appearance in the KB
      
      clauses: a set of clause from the KB
      ordered_list: a list with symbols order by the increasing appearance in the KB
  '''

  # dictionary to store the number of appearance of each element
  elements = dict()
  for obj in objects:
    elements[obj] = 0

  # loop through the KB to calculate the appearance of the symbols
  for clause in KB:
    for symbol in clause:
      elements[symbol] += 1
  
  # sort the dictionary and return the sorted_list
  sorted_elements = sorted(elements.items(), key=operator.itemgetter(1))
  sorted_symbols = [element[0] for element in sorted_elements]
  return sorted_symbols


def deg_heuristic(curr_symbols):
  global sorted_symbols
  ''' pop, remain = deg_heuristic(curr_symbols)
      Takes in the current set of symbols in the KB (those are not yet determined)
      and pop out the symbols with highest appearance in the KB
      
      curr_symbols: the current set of symbols
  '''

  # moving backward as sorted_symbols is increasing 
  for i in range(len(sorted_symbols) - 1, -1, -1):
    # find the first occurence of the symbol in curr_symbol:
    # the one with highest appearance in KB among the curr_symbols
    if sorted_symbols[i] in curr_symbols:
      list_curr_symbols = list(curr_symbols)
      pop_index = list_curr_symbols.index(sorted_symbols[i])
      pop_symbol = list_curr_symbols.pop(pop_index)
      return pop_symbol, set(list_curr_symbols)


def unit_clause(KB, symbols):
  """
  unit = unit_clause(KB, symbols)
  Input the KB and the symbols, return all the unit clauses
  Return -1 if there is a conflict: KB unsatisfiable

  KB: set of clauses
  symbols: all the available symbols 
  unit: set of unit clauses
  """

  unit = set()
  # pos_unit and neg_unit are to check if there are conflicts in unit clauses: e.g {C} and {-C}
  pos_unit = set()
  neg_unit = set()
  for clause in KB:
    # unit clause
    if len(clause) == 1:
      for symbol in clause:
        unit.add(symbol)
        # use to check conflict
        if symbol.sign:
          pos_unit.add(symbol)
        else:
          neg_unit.add(symbol)

  # if there is a unit clause both positive and negative
  if len(pos_unit.intersection(neg_unit)) > 0:
    return -1
  return unit


def pure_symbol(KB, symbols):
  """
  sorted_pure_symbols =pure_symbol(KB, symbols)
  Input the KB and the symbols, return all the unit clauses
  Return -1 if there is a conflict: KB unsatisfiable

  KB: set of clauses
  symbols: all the available symbols 
  unit: set of unit clauses
  """

  # pure clause = the XOR of positive set and negative set
  positive = set()
  negative = set()
  for clause in KB:
    for symbol in clause:
      if symbol.sign:
        positive.add(symbol)
      else:
        negative.add(symbol)
  pure = positive ^ negative
  
  # sort the elements to pop out the most appeared pure symbols
  elements = dict()
  for obj in objects:
    elements[obj] = 0

  for clause in KB:
    for symbol in clause:
      elements[symbol] += 1
  
  # a dictionary of pure symbols and its frequency
  pure_elements = dict()
  while pure:
    obj = pure.pop()
    pure_elements[obj] = elements[obj]

  # sort the pure_elements and output the list of pure symbols ordered by its frequency  
  sorted_pure_elements = sorted(pure_elements.items(), key=operator.itemgetter(1))
  sorted_pure_symbols = [element[0] for element in sorted_pure_elements]
  return sorted_pure_symbols
  

#######################################n
# for the test case only: similar functions have been implemented above
def degree_heuristic(KB, symbols, value):
  """
  S = degree_heuristic(clauses, symbols, model):
      clauses: The KB with a list of clauses
      symbols: A set of the current literals (we are only interested in the literal, not the sign)
      model: A dictionary giving the truth values assigned for the model
      S: A literal object representing the unassigned 
          symbol that appears the most across all clauses (should be positive)  
  """
  # the function to pass the test case 
  # similar to the above function
  elements = dict()

  # implement the sorting again 
  for symbol in symbols:
    elements[symbol] = 0
  for clause in KB:
    for symbol in clause:
      elements[symbol] += 1
  
  sorted_elements = sorted(elements.items(), key=operator.itemgetter(1))
  sorted_symbols = [element[0] for element in sorted_elements]
  # return the most frequent 
  return sorted_symbols[-1]


def find_unit_clause(KB, symbols, value):
  """
  U = find_unit_clause(clauses, symbols, model):
      clauses: The KB with a list of clauses
      symbols: A set of the current literals (we are only interested in the literal, not the sign)
      model: A dictionary giving the truth values assigned for the model
      U:  A unit clause (currently unassigned in model)
  """
  unit = set()
  for clause in KB:
    if len(clause) == 1:
      for symbol in clause:
        unit.add(symbol)
  elements = sort_symbol(KB)

  # pop the most frequent element
  while elements:
    if elements[-1] in unit:
      return elements[-1]
    else:
      elements.pop(-1)
  return False


def find_pure_symbol(KB, symbols, value):
  """
  P = find_pure_symbol(clauses, symbols, model):
      clauses: The KB with a list of clauses
      symbols: A set of the current literals (we are only interested in the literal, not the sign)
      model: A dictionary giving the truth values assigned for the model
      P: A pure symbol (currently unassigned in model)
  """
  positive = set()
  negative = set()
  for clause in KB:
    for symbol in clause:
      if symbol.sign:
        positive.add(symbol)
      else:
        negative.add(symbol)
  pure = positive ^ negative
  elements = sort_symbol(KB)
  
  # pop the most frequent elemet
  while elements:
    if elements[-1] in pure:
      return elements[-1]
    else:
      elements.pop(-1)
  return False

<h1>Question 3</h1>

Implement your KB from Russell & Norvig in CNF by filling in the sets inside the list <code>KB</code> below.  Ensure that your KB is in a list of set format as stated in the assignment instructions.

In [4]:
A = Literal('A')
B = Literal('B')
C = Literal('C')
D = Literal('D')
E = Literal('E')
F = Literal('F')

KB = [{-A, B, E}, {-B, A}, {-E, A}, {-E, D}, {-C, -F, -B}, {-E, B}, {-B, F}, {-B, C}]
objects = list(get_symbol(KB))
unit_clauses = unit_clause(KB, objects)
pure_symbols = pure_symbol(KB, objects)
sorted_symbols = sort_symbol(KB)
DPLL_Satisfiable(KB)

(True, {A: False, B: False, C: True, D: True, E: False, F: True})

<h1>Extensions</h1>

1. Complete the degree heuristic function by filling in the placeholder in Q2.
2. Complete the pure symbol and unit clause heuristics by filling in the placeholder in Q2.
3. Modify the pure symbol heuristic to choose the pure symbol that occurs in the most number of clauses.

<h1>Basic Functionality Tests</h1>

All of the tests in this section must be passed for the code to be considered fully functional.  Other unseen test cases will be used after submission to further verify functionality

In [0]:
# This section will test the correct definition of the literal class

# Define some literals
A = Literal('A')
B = Literal('B')
C = Literal('C')
D = Literal('D')
E = Literal('E')
F = Literal('F')


# Test the name attribute works correctly
assert(A.name == 'A')

# Test that negation works correctly
assert(not (-C).sign)

In [0]:
# This section will test that the KB has been correctly converted to CNF by performing 
# tests over all possible models
import itertools
symbols = {A,B,C,D,E,F}
symbol_list = [A,B,C,D,E,F]

def evaluate_russell_norvig_KB(model):
    # Manually evaluate the KB using the model
    s = list()
    s.append(model[A] == (model[B] or model[E]))
    s.append(model[E] <= model[D])
    s.append((model[C] and model[F]) <= (not model[B]))
    s.append(model[E] <= model[B])
    s.append(model[B] <= model[F])
    s.append(model[B] <= model[C])
    return all(s)

def evaluate_KB(KB, model):
    model_true = True
    model_name_dict = {l.name: model[l] for l in model}
    for clause in KB:
        evaluation = {l.sign == model_name_dict[l.name] for l in clause if l.name in model_name_dict}
        model_true = model_true and any(evaluation)
    return model_true

all_models = list(itertools.product([False, True], repeat=6))
for i in range(0, len(all_models)):
    # Test all truth values
    test_model = {symbol_list[j]: all_models[i][j] for j in range(0,6)}
    # Test the model
    assert(evaluate_russell_norvig_KB(test_model) == evaluate_KB(KB, test_model))

In [0]:
# This section will test the basic working of DPLL
# Satisfiable model
test_KB = [{A, C},{-A, C}, {B, -C}]
objects = list(get_symbol(test_KB))
unit_clauses = unit_clause(test_KB, objects)
pure_symbols = pure_symbol(test_KB, objects)
sorted_symbols = sort_symbol(test_KB)
t, model = DPLL_Satisfiable(test_KB)
assert(evaluate_KB(test_KB, model))
assert(t)

# Unsatisfiable model
test_KB = [{A, C},{-A, C}, {-C}]
objects = list(get_symbol(test_KB))
unit_clauses = unit_clause(test_KB, objects)
pure_symbols = pure_symbol(test_KB, objects)
sorted_symbols = sort_symbol(test_KB)
t, model = DPLL_Satisfiable(test_KB)
assert(not t)

In [0]:
# This will test DPLL on the KB from Russell & Norvig

'''
# S1:A⇔(B∨E). 
<=> (A=>(BvE)) ∧ ((BvE)=>A)
<=> ((BvE)v-A) ∧ (Av-(BvE))
<=> (BvEv-A) ∧ (Av(-B∧-E))
<=> (BvEvA) ∧ (Av-B) ∧ (Av-E)

# S2:E⇒D.
<=> Dv-E

# S3:C∧F⇒¬B.
<=> -Bv-(C∧F)
<=> -Bv-Cv-F

# S4:E⇒B.
<=> Bv-E

# S5:B⇒F.
<=> Fv-B

# S6:B⇒C
<=> Cv-B

'''

KB = [{B,E,A}, {A,-B},{A,-E},{D,-E},{-B,-C,-F},{B,-E},{-B,F},{-B,C}]
objects = [A,B,C,D,E,F]
sorted_symbols = sort_symbol(KB)
t, model = DPLL_Satisfiable(KB)

# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model))   

<h1>Extension Tests</h1>

This section will test that the degree heuristic, pure symbol and unit clause heuristics, picking the most frequently used symbol.

In [0]:
# Degree Heuristic Test
KB = [{B,E,A}, {A,-B},{A,-E},{D,-E},{-B,-C,F},{B,-E},{-B,F},{-B,C}]
S = degree_heuristic(KB, symbols, dict())
assert(S.name == 'B')

In [0]:
# Pure Symbol Test
P = find_pure_symbol([{A,B,C},{A,-D},{A,D}], {A,B,C,D}, dict())
assert(P.name == 'A')

In [0]:
# Unit Clause Test
U = find_unit_clause([{A,B,C},{-C},{A, D}], {A,B,C,D}, dict())
assert(U.name == 'C')